Экономика городов. DBScan кластеризация.

##### Шарифов Фаррух Рустамович N33472

In [19]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import os

from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [20]:
os.chdir("C:/Users/Farrukh99/Desktop/subjects/3/Data processing/lecture 3/Lab1")

In [21]:
df = pd.read_csv("Econom_Cities_data.csv", sep=';', decimal=',', index_col='City')

In [22]:
print(df)

                Work  Price  Salary
City                               
Amsterdam       1714   65.6    49.0
Athens          1792   53.8    30.4
Bogota          2152   37.9    11.5
Bombay          2052   30.3     5.3
Brussels        1708   73.8    50.5
Buenos_Aires    1971   56.1    12.5
Cairo          -9999   37.1 -9999.0
Caracas         2041   61.0    10.9
Chicago         1924   73.9    61.9
Copenhagen      1717   91.3    62.9
Dublin          1759   76.0    41.4
Dusseldorf      1693   78.5    60.2
Frankfurt       1650   74.5    60.4
Geneva          1880   95.9    90.3
Helsinki        1667  113.6    66.6
Hong_Kong       2375   63.8    27.8
Houston         1978   71.9    46.3
Jakarta        -9999   43.6 -9999.0
Johannesburg    1945   51.1    24.0
Kuala_Lumpur    2167   43.5     9.9
Lagos           1786   45.2     2.7
Lisbon          1742   56.2    18.8
London          1737   84.2    46.2
Los_Angeles     2068   79.8    65.2
Luxembourg      1768   71.1    71.1
Madrid          1710   93.8 

Для замены ошибочных значений, таких как work и solary в городах jakarta и ciaro, будем заполнять средным значением по выборке!

In [23]:
work = df['Work'].copy()
work['Cairo'] = None
work['Jakarta'] = None

mean = int(work.mean())
df['Work'] = work.fillna(mean)

salary = df['Salary'].copy()
salary['Cairo'] = None
salary['Jakarta'] = None

mean_s = int(salary.mean())
df['Salary'] = salary.fillna(mean_s)

df

,Work,Price,Salary
City,,,
Amsterdam,1714.0,65.6,49.0
Athens,1792.0,53.8,30.4
Bogota,2152.0,37.9,11.5
Bombay,2052.0,30.3,5.3
Brussels,1708.0,73.8,50.5
Buenos_Aires,1971.0,56.1,12.5
Cairo,1879.0,37.1,39.0
Caracas,2041.0,61.0,10.9
Chicago,1924.0,73.9,61.9


Стандартизация данных.

Так как они отличаются по смыслу и следовательно их значения различны на порядок.

In [24]:
from sklearn import preprocessing

norm = preprocessing.StandardScaler()
X = norm.fit_transform(df)
X = pd.DataFrame(X, index=df.index, columns=df.columns)

print(X)

                    Work     Price    Salary
City                                        
Amsterdam      -0.982634 -0.151250  0.395343
Athens         -0.520566 -0.698647 -0.380569
Bogota          1.612052 -1.436242 -1.168996
Bombay          1.019658 -1.788804 -1.427633
Brussels       -1.018177  0.229145  0.457917
Buenos_Aires    0.539819 -0.591951 -1.127280
Cairo          -0.005183 -1.473354 -0.021814
Caracas         0.954495 -0.364642 -1.194025
Chicago         0.261394  0.233784  0.933476
Copenhagen     -0.964862  1.040964  0.975191
Dublin         -0.716056  0.331203  0.078304
Dusseldorf     -1.107036  0.447177  0.862559
Frankfurt      -1.361766  0.261618  0.870902
Geneva          0.000740  1.254356  2.118201
Helsinki       -1.261059  2.075453  1.129539
Hong_Kong       2.933091 -0.234751 -0.489030
Houston         0.581287  0.141005  0.282711
Jakarta        -0.005183 -1.171821 -0.021814
Johannesburg    0.385797 -0.823899 -0.647549
Kuala_Lumpur    1.700911 -1.176460 -1.235741
Lagos     

Попробуем обучить модель используя значения по умолчанию

In [25]:
dbscan = DBSCAN(eps=0.5, metric='euclidean', min_samples=5)
dbscan.fit(X)
dbscan.labels_

array([ 0, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  0, -1, -1,  0, -1, -1,  0,  0, -1, -1, -1, -1,
       -1,  0, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1],
      dtype=int64)

Создадим таблицу частот

In [26]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1 37]
 [ 0 11]]


37 выбросов и 1 кластер из 11 элементов нам не интересны

In [27]:
dbscan = DBSCAN(eps=5, metric='euclidean', min_samples=5)

dbscan.fit(X)

DBSCAN(eps=5)

In [28]:
#  Таблица частот 

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0 48]]


Уменьшаем eps и min_samples, так как все объекты оказались в одном кластере.

In [29]:
dbscan = DBSCAN(eps=0.8, metric='euclidean', min_samples=5)

dbscan.fit(X)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  9]
 [ 0 18]
 [ 1 21]]


In [30]:
dbscan = DBSCAN(eps=0.8, metric='euclidean', min_samples=2.1)

dbscan.fit(X)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 21]]


два раза попробовали, построить модель лучше, не получилось.

Сравним с результатами иерархического кластерного анализа этих же данных

In [31]:
link = linkage(df, 'ward', 'euclidean')
df['cluster'] = fcluster(link, 2, criterion='maxclust')
df['dbscan'] = dbscan.labels_
df.head()

,Work,Price,Salary,cluster,dbscan
City,,,,,
Amsterdam,1714.0,65.6,49.0,2,0
Athens,1792.0,53.8,30.4,2,1
Bogota,2152.0,37.9,11.5,1,1
Bombay,2052.0,30.3,5.3,1,1
Brussels,1708.0,73.8,50.5,2,0


In [32]:
print (df['dbscan'].value_counts())

 1    21
 0    19
-1     8
Name: dbscan, dtype: int64


In [33]:
tab = pd.crosstab(df['dbscan'], df['cluster'])

print(tab)


cluster  1   2
dbscan        
-1       2   6
 0       1  18
 1       8  13


Отличия моделей более чем заметны. Посмотрим на значения кластеров.

In [34]:
df.groupby('dbscan').mean()

,Work,Price,Salary,cluster
dbscan,,,,
-1,1900.375000,99.925000,61.262500,1.750000
0,1792.000000,77.526316,55.157895,1.947368
1,1951.571429,49.185714,17.095238,1.619048


#### Вывод
Не смотря на присутсвие выбросов, модель построенная на методе dbscan мне импонирует, так как необходимость работать больше из-за низких зарплат для второго клаcтера (dbscan = 1), более выражена чем в модели k-средних.